<h1 align="center"><a href="https://github.com/sborquez/volcano-seismic_classifier">Volcano-Seismic Classifier</a> - Automatic classification of seismic signals from Llaima volcano (Chile).</h1>

<center>
<br>
<img align="center" src="images/utfsm.png" width="50%"/>

<h2 align="center">K-Fold Cross-Validation</h2>
</center>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez@sansano.usm.cl">sebastian.borquez@sansano.usm.cl</a> - utfsm - Jul 2021.</i>
</center>


In [1]:
import data
import models
import performance

from tools import *

In [2]:
!pip install wandb --upgrade -qqq

In [3]:
#os.environ['WANDB_MODE'] = 'online'

In [4]:
project="volcano-seismic"
dataset_artifact_name = 'llaima-preprocessed-1d:latest'
labels = ["lp", "tc", "tr", "vt"] 

# Table of Content

* [K-Fold Cross-Validation](#crossvalidation)
* [Multi Layer Perceptron](#mlp)
* [SeismicNet](#seismic)
* [SeismicNet + Focal Loss](#seismic_fl)
* [Multi-Head](#multihead)
* [Multi-Head + Focal Loss](#multihead_fl)
* [LSTM](#lstm)
* [LSTM + Focal Loss](#lstm_fl)
* [GRU](#gru)
* [GRU + Focal Loss](#gru_fl)
* [Time Series Transformer](#transformer)
* [Time Series Transformer + Focal Loss](#transformer_fl)
* [References](#references)

<a id="crossvalidation"><a/>

# K-Fold Cross-Validation

![Kfold](images/02/cross_validation.png)



## Model Training

In [5]:
def evaluate_model(run, model, X_test, y_test, num_classes, labels,
                     metrics, plots, top_k, **kwargs):
    run_id=kwargs.get("run_id", None)
    test_index=kwargs.get("test_index", None)
    y_prob = model.predict(X_test)
    y_test = y_test.argmax(axis=-1) if num_classes > 2 else y_test
    model_metrics, model_metrics_tables = performance.get_metrics(y_test, y_prob, metrics=metrics, labels=labels, num_classes=num_classes, run_id=run_id)
    model_plots = performance.get_plots(X_test, y_test, y_prob, plots=plots, labels=labels, num_classes=num_classes)
    model_highlights= performance.get_highlights(X_test, y_test, y_prob, test_index, labels, top_k=top_k)
    # Log to WandB
    ########################################
    run.log(
        dict(
            **model_metrics,
            **model_metrics_tables,
            **model_plots,
            **model_highlights
        )
    )
    return y_prob, model_metrics

def training_fold(worker_data):
    project=worker_data.project
    run_name = "{}-{}".format(worker_data.sweep_run_name, worker_data.num)
    config = worker_data.config
    dataset_artifact_name = worker_data.dataset_artifact_name
    # this block enables GPU enabled multiprocessing
    core_config = tf.compat.v1.ConfigProto()
    core_config.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=core_config)
    tf.compat.v1.keras.backend.set_session(session)
    # Setup WandB
    with wandb.init(project=project, group=worker_data.sweep_id, job_type="model-training", 
                    name=run_name, config=config) as run:    
        # Model builder configuration
        ########################################
        model_hyperparameters = config["model_hyperparameters"]
        training_configuration = config["training_configuration"]
        evaluation_configuration = config["evaluation_configuration"]
        verbosity = training_configuration["verbosity"]
        builder = models.builders.get(model_hyperparameters["builder"])
        hyper_parameters = dict(model_hyperparameters)
        del hyper_parameters["builder"]
        # Load Data
        ########################################
        labels = worker_data.labels
        num_classes = worker_data.num_classes
        # ✔️ declare which artifact we'll be using
        dataset_artifact = run.use_artifact(dataset_artifact_name)
        # 📥 if need be, download the artifact
        dataset = dataset_artifact.download()
        X,y = data.read_dataset(dataset, name="processed")
        # Split folds
        train_index = sklearn.utils.shuffle(worker_data.train_index) # I don't know why is not shuffle
        X_train, y_train = X[train_index], y[train_index]
        test_index = worker_data.test_index
        X_test, y_test = X[test_index], y[test_index]
        if verbosity > 0:
            print('Num clases:', num_classes)
            print('Labels:', labels, type(labels))
            print('X_train shape:', X_train.shape)
            print('y_train shape:', y_train.shape)
            print('X_test shape:', X_test.shape)
            print('y_test shape:', y_test.shape)
        # Training configuration
        ########################################
        epochs = training_configuration["epochs"]
        batch_size = training_configuration["batch_size"]
        loss =  models.get_loss(**training_configuration["loss"])
        optimizer =  models.get_optimizer(**training_configuration["optimizer"])
        callbacks = models.get_callbacks(labels=labels, verbosity=verbosity,
                                        **training_configuration["callbacks"])
        # Load Model
        ########################################
        model = builder(**hyper_parameters)
        # Compile the model
        model.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=['categorical_accuracy' if num_classes > 2 else 'binary_accuracy']
        )
        assert model.predict(2*np.random.random((batch_size,6000,1)) - 1).shape == (batch_size, num_classes), "Incorrect output shape."
        if verbosity > 1:
            model.summary()
        # Train fold
        ########################################
        print(f'Training for fold {worker_data.num+1} ...')
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                  epochs=epochs, verbose=verbosity, batch_size=batch_size,
                  callbacks=callbacks
        )
        # Evaluation
        ########################################
        # Evaluation configuration TODO: use model.evaluate(..., callbacks=WandBCallback?)
        # load best mode
        best_model = model
        best_model.load_weights(Path(run.dir)/"model-best.h5")
        y_prob, model_metrics = evaluate_model(
            run, best_model, X_test, y_test, num_classes, labels, **evaluation_configuration
        )
        run.join()
    # Finally, close sessions
    session.close()
    tf.keras.backend.clear_session() 
    print(f"Worker {worker_data.num} finished")
    return WorkerDoneData(num=worker_data.num, X_test=X_test, y_test=y_test, 
                          y_prob=y_prob, metrics=model_metrics)

def k_fold_cross_validation(project, experiment_name, dataset_artifact_name, labels,
            model_hyperparameters, training_configuration, evaluation_configuration,
            k=10, seed=None):
    # K-fold Parameters
    ########################################
    num_classes = len(labels)
    num_folds = k
    # Setup Cross-Validation WandB Runs Group
    ##########################################
    cv_config = {
        "model_hyperparameters": model_hyperparameters,
        "training_configuration": training_configuration,
        "evaluation_configuration": evaluation_configuration,
        "seed": seed
    }
    sweep_id = wandb.util.generate_id() #sweep_run.id
    sweep_run_name = experiment_name # or sweep_run.id or "unknown"    
    with wandb.init(project=project, id=sweep_id, name=sweep_run_name, job_type="cross-validation", config=cv_config) as sweep_run:
        project_url = sweep_run.get_project_url()
        sweep_group_url = "{}/groups/{}".format(project_url, sweep_id)
        sweep_run.notes = sweep_group_url
        print("*" * 40)
        print("Sweep Group URL: ", sweep_group_url)
        print("*" * 40)  
        # Load Data
        ########################################
        # declare which artifact we'll be using
        dataset_artifact = sweep_run.use_artifact(dataset_artifact_name)
        # 📥 if need be, download the artifact
        dataset = dataset_artifact.download()
        X,y = data.read_dataset(dataset, name="processed")
        print('Num clases:', num_classes)
        print('Labels:', labels)
        print('X shape:', X.shape)
        print('y shape:', y.shape)
    
    # Split 10-Folds
    ########################################
    # Define the K-fold Cross Validator
    kfold = sklearn.model_selection.StratifiedKFold(
        n_splits=num_folds, shuffle=True, random_state=seed
    )
    # K-fold Cross Validation model evaluation
    evaluations = {}
    #folds = []
    for num, (train_index, test_index) in enumerate(kfold.split(X, y.argmax(axis=-1))):
        print('------------------------------------------------------------------------')
        print(f'Worker for fold {num+1} ...')
        config = {
            "model_hyperparameters" : model_hyperparameters,
            "training_configuration": training_configuration,
            "evaluation_configuration": evaluation_configuration,
            "fold": num
        }
        config.update(cv_config)
        # start worker
        worker_data = WorkerInitData(
                num=num,
                project=project,
                sweep_id=sweep_id,
                sweep_run_name=sweep_run_name,
                config=config,
                dataset_artifact_name=dataset_artifact_name,
                train_index=train_index,
                test_index=test_index,
                labels=labels,
                num_classes=num_classes
        )
        result = training_fold(worker_data)
        evaluations[result.num] = result.metrics
        #folds.append(client.submit(training_fold, worker_data))
    #results = client.gather(folds)
    #evaluations = {result.num: result.metrics for result in results}
    # Table of fold metrics and mean/variance row
    ########################################
    # folds
    metrics_df = pd.DataFrame(evaluations).T.reset_index().rename(columns={"index": "id"})
    metrics_df["id"] = metrics_df["id"].apply(lambda f: f'fold {f}')
    # mean
    metrics_mean = metrics_df.mean()
    metrics_mean["id"] = "mean"
    metrics_df = metrics_df.append(metrics_mean, ignore_index=True)
    # std
    metrics_std = metrics_df.std()
    metrics_std["id"] = "std"
    metrics_df = metrics_df.append(metrics_std, ignore_index=True)
    # Log Table, mean and variance of all metrics
    ########################################
    metrics_mean_dict = {f"mean {m}": v for m,v in metrics_mean.drop(labels=["id"]).iteritems()}
    metrics_std_dict = {f"std {m}": v for m,v in metrics_mean.drop(labels=["id"]).iteritems()}
    with wandb.init(project=project, id=sweep_id, name=sweep_run_name, job_type="cross-validation", config=cv_config, resume="allow") as sweep_run:
        sweep_id = sweep_run.id
        ## Log to WandB
        sweep_run.log(dict(
            evaluation = wandb.Table(dataframe=metrics_df),
            **metrics_mean_dict,
            **metrics_std_dict,
        ))
    return metrics_df

<a id="mlp"><a/>

# Multi Layer Perceptron (Test model)

In [6]:
experiment_name="MultiLayerPerceptron"

model_hyperparameters = {
    "builder": "mlp_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "layers_units": [375],
    "dropout_rate": 0.75
}

training_configuration = {
    "epochs": 200,
    "batch_size": 128,
    "verbosity": 0,
    "loss": {
        #"name": "categorical_crossentropy",
        #"parameters": {}
        "name": "categorical_focal_loss",
        "parameters": {
            "gamma": 2.0,
            "alpha": np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
    "callbacks": {
        "early_stop": 5
    }
}

evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_mlp = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [13]:
evaluation_metrics_mlp

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.908333,0.091667,0.786365,0.929766,0.716285,0.739447,0.719514,125.000000,127.000000,...,0.510204,0.548387,0.916667,0.861111,0.919444,0.936111,0.892857,0.835526,0.632911,0.596491
1,fold 1,0.893056,0.106944,0.747795,0.921092,0.730915,0.734402,0.679597,128.000000,107.000000,...,0.551020,0.677419,0.897222,0.827778,0.900000,0.947222,0.873720,0.775362,0.600000,0.688525
2,fold 2,0.906685,0.093315,0.776853,0.931728,0.771284,0.770522,0.721247,124.000000,114.000000,...,0.673469,0.700000,0.894150,0.860724,0.927577,0.944290,0.867133,0.820144,0.717391,0.677419
3,fold 3,0.923398,0.076602,0.821730,0.944458,0.809306,0.812909,0.771261,129.000000,118.000000,...,0.693878,0.766667,0.933148,0.877437,0.913649,0.969359,0.914894,0.842857,0.686869,0.807018
4,fold 4,0.873259,0.126741,0.688520,0.908918,0.679963,0.680545,0.622568,122.000000,103.000000,...,0.530612,0.566667,0.885794,0.807799,0.860724,0.938719,0.856140,0.749091,0.509804,0.607143
5,fold 5,0.912256,0.087744,0.812181,0.934440,0.778381,0.790098,0.736647,124.000000,116.000000,...,0.755102,0.633333,0.899721,0.860724,0.930362,0.958217,0.873239,0.822695,0.747475,0.716981
6,fold 6,0.906685,0.093315,0.789168,0.930164,0.761086,0.771939,0.718478,126.000000,117.000000,...,0.530612,0.766667,0.913649,0.849582,0.894150,0.969359,0.890459,0.812500,0.577778,0.807018
7,fold 7,0.894150,0.105850,0.733960,0.923046,0.726325,0.727260,0.682489,129.000000,110.000000,...,0.448980,0.733333,0.922006,0.830084,0.866295,0.958217,0.902098,0.782918,0.478261,0.745763
8,fold 8,0.876045,0.123955,0.739904,0.905980,0.665310,0.686215,0.623694,129.000000,101.000000,...,0.510204,0.483871,0.866295,0.802228,0.891365,0.944290,0.843137,0.739927,0.561798,0.600000
9,fold 9,0.902507,0.097493,0.791919,0.928031,0.736777,0.754042,0.706823,130.000000,111.000000,...,0.591837,0.612903,0.894150,0.860724,0.896936,0.958217,0.872483,0.816176,0.610526,0.716981


<a id="SeismicNet"><a/>

# SeismicNet

In [7]:
experiment_name="SeismicNet"

model_hyperparameters = {
    "builder": "seismicnet_builder",
    "input_shape": (6000,1),
    "num_classes":4,
}

training_configuration = {
    "epochs": 200,
    "batch_size": 128,
    "verbosity": 0,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {}
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
    "callbacks": {
        "early_stop": 5
    }
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_seismicnet = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [9]:
evaluation_metrics_seismicnet

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.976389,0.023611,0.936857,0.982878,0.934966,0.935823,0.929555,126.000000,143.000000,...,0.979592,0.838710,0.969444,0.969444,0.991667,0.975000,0.958175,0.962963,0.969697,0.852459
1,fold 1,0.988889,0.011111,0.966190,0.991982,0.960108,0.963051,0.966790,127.000000,149.000000,...,1.000000,0.870968,0.983333,0.991667,1.000000,0.980556,0.976923,0.990033,1.000000,0.885246
2,fold 2,0.980501,0.019499,0.951626,0.985189,0.929230,0.939341,0.941419,128.000000,145.000000,...,1.000000,0.766667,0.974930,0.974930,1.000000,0.972145,0.966038,0.969900,1.000000,0.821429
3,fold 3,0.976323,0.023677,0.946061,0.982433,0.943354,0.944384,0.929110,127.000000,143.000000,...,0.877551,0.966667,0.969359,0.969359,0.974930,0.991643,0.958491,0.962963,0.905263,0.950820
4,fold 4,0.986072,0.013928,0.954329,0.990436,0.968565,0.960417,0.958468,126.000000,147.000000,...,0.959184,0.966667,0.980501,0.986072,0.994429,0.983287,0.972973,0.983278,0.979167,0.906250
5,fold 5,0.990251,0.009749,0.969361,0.993146,0.977484,0.973149,0.970893,128.000000,147.000000,...,0.979592,0.966667,0.986072,0.988858,0.997214,0.988858,0.980843,0.986577,0.989691,0.935484
6,fold 6,0.991643,0.008357,0.981644,0.993664,0.963311,0.971786,0.974934,131.000000,147.000000,...,1.000000,0.866667,0.988858,0.991643,1.000000,0.986072,0.984962,0.989899,1.000000,0.912281
7,fold 7,0.983287,0.016713,0.946829,0.988337,0.948897,0.947774,0.950055,127.000000,146.000000,...,0.979592,0.866667,0.974930,0.986072,0.997214,0.974930,0.965779,0.983165,0.989691,0.852459
8,fold 8,0.983287,0.016713,0.947479,0.988583,0.962677,0.953554,0.950345,129.000000,142.000000,...,0.938776,0.967742,0.986072,0.974930,0.991643,0.980501,0.980989,0.969283,0.968421,0.895522
9,fold 9,0.981894,0.018106,0.943182,0.987354,0.941424,0.942131,0.945956,127.000000,146.000000,...,0.938776,0.870968,0.980501,0.983287,0.988858,0.974930,0.973180,0.979866,0.958333,0.857143


<a id="seismic_fl"><a/>

# SeismicNet + Focal Loss

In [10]:
#Focal Loss
alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

In [11]:
experiment_name  = "SeismicNetFocalLoss"

model_hyperparameters = {
    "builder": "seismicnet_builder",
    "input_shape": (6000,1),
    "num_classes":4,
}

training_configuration = {
    "epochs": 200,
    "batch_size": 128,
    "verbosity": 0,
    "loss": {
        "name": "categorical_focal_loss",
        "parameters": {
            "alpha": alpha,
            "gamma": 2.0
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
    "callbacks": {
        "early_stop": 5
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_seismicnet_fl = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [13]:
evaluation_metrics_seismicnet_fl

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.976389,0.023611,0.938179,0.982692,0.937929,0.938048,0.929540,126.000000,143.0,...,0.959184,0.870968,0.975000,0.963889,0.988889,0.977778,0.965517,0.956522,0.959184,0.870968
1,fold 1,0.988889,0.011111,0.977952,0.991505,0.954183,0.964941,0.966696,129.000000,148.0,...,1.000000,0.838710,0.977778,0.994444,1.000000,0.983333,0.969925,0.993289,1.000000,0.896552
2,fold 2,0.981894,0.018106,0.946328,0.987147,0.953645,0.949730,0.945956,126.000000,145.0,...,0.979592,0.900000,0.972145,0.980501,0.997214,0.977716,0.961832,0.976431,0.989691,0.870968
3,fold 3,0.977716,0.022284,0.949085,0.983098,0.934952,0.940936,0.933018,127.000000,146.0,...,0.857143,0.933333,0.974930,0.972145,0.977716,0.986072,0.965779,0.966887,0.913043,0.918033
4,fold 4,0.984680,0.015320,0.948206,0.989676,0.966656,0.955936,0.954368,125.000000,147.0,...,0.959184,0.966667,0.977716,0.986072,0.994429,0.980501,0.968992,0.983278,0.979167,0.892308
5,fold 5,0.987465,0.012535,0.959292,0.991777,0.974359,0.966523,0.962699,129.000000,144.0,...,0.979592,0.966667,0.991643,0.980501,0.986072,0.991643,0.988506,0.976271,0.950495,0.950820
6,fold 6,0.983287,0.016713,0.950301,0.987961,0.964710,0.956776,0.950167,123.000000,147.0,...,1.000000,0.933333,0.974930,0.977716,1.000000,0.980501,0.964706,0.973510,1.000000,0.888889
7,fold 7,0.981894,0.018106,0.949079,0.986904,0.940564,0.944701,0.945766,127.000000,146.0,...,0.979592,0.833333,0.969359,0.986072,0.997214,0.974930,0.958491,0.983165,0.989691,0.847458
8,fold 8,0.976323,0.023677,0.926049,0.983971,0.947418,0.934911,0.929817,127.000000,140.0,...,0.938776,0.935484,0.977716,0.966574,0.988858,0.972145,0.969466,0.958904,0.958333,0.852941
9,fold 9,0.981894,0.018106,0.956301,0.986641,0.942301,0.948928,0.945862,131.000000,142.0,...,0.938776,0.870968,0.986072,0.972145,0.986072,0.983287,0.981273,0.965986,0.948454,0.900000


<a id="multihead"><a/>

# Multi-Head

In [7]:
experiment_name  = "Multi-Head"

model_hyperparameters = {
    "builder": "multihead_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "filters": [32, 32, 32]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 1,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {}
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_multihead = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [9]:
evaluation_metrics_multihead

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.969444,0.030556,0.933671,0.976622,0.912650,0.922643,0.908289,125.00000,143.000000,...,0.897959,0.838710,0.963889,0.955556,0.980556,0.977778,0.950570,0.947020,0.926316,0.866667
1,fold 1,0.984722,0.015278,0.953380,0.989376,0.948919,0.950804,0.954408,128.00000,146.000000,...,1.000000,0.838710,0.980556,0.986111,0.994444,0.977778,0.973384,0.983165,0.980000,0.866667
2,fold 2,0.967967,0.032033,0.903187,0.978015,0.916246,0.909399,0.904645,124.00000,141.000000,...,0.938776,0.833333,0.955432,0.969359,0.980501,0.966574,0.939394,0.962457,0.929293,0.806452
3,fold 3,0.972145,0.027855,0.930558,0.979920,0.931702,0.930932,0.916697,128.00000,141.000000,...,0.836735,0.966667,0.974930,0.961003,0.961003,0.991643,0.966038,0.952703,0.854167,0.950820
4,fold 4,0.981894,0.018106,0.957616,0.986341,0.953221,0.955269,0.945762,125.00000,147.000000,...,0.938776,0.933333,0.977716,0.974930,0.986072,0.988858,0.968992,0.970297,0.948454,0.933333
5,fold 5,0.977716,0.022284,0.934777,0.984399,0.948418,0.941306,0.933597,126.00000,143.000000,...,0.938776,0.933333,0.980501,0.966574,0.980501,0.983287,0.972973,0.959732,0.929293,0.903226
6,fold 6,0.986072,0.013928,0.962542,0.989818,0.959139,0.960725,0.958335,128.00000,146.000000,...,0.979592,0.900000,0.988858,0.977716,0.991643,0.986072,0.984615,0.973333,0.969697,0.915254
7,fold 7,0.974930,0.025070,0.931257,0.982056,0.934790,0.932100,0.925029,126.00000,144.000000,...,0.877551,0.933333,0.969359,0.972145,0.977716,0.980501,0.958175,0.966443,0.914894,0.888889
8,fold 8,0.974930,0.025070,0.939150,0.981272,0.919765,0.928417,0.924798,129.00000,143.000000,...,0.857143,0.870968,0.977716,0.966574,0.977716,0.977716,0.969925,0.959732,0.913043,0.870968
9,fold 9,0.987465,0.012535,0.980533,0.990157,0.952470,0.965277,0.962418,131.00000,145.000000,...,0.959184,0.870968,0.986072,0.983287,0.991643,0.988858,0.981273,0.979730,0.969072,0.931034


<a id="multihead_fl"><a/>

# Multi Head + Focal Loss

In [6]:
experiment_name  = "Multi-HeadFocalLoss"

#Focal Loss
alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

model_hyperparameters = {
    "builder": "multihead_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "filters": [32, 32, 32]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 1,
    "loss": {
        "name": "categorical_focal_loss",
        "parameters": {
            "alpha": alpha,
            "gamma": 2.0
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_multihead_fl = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [10]:
evaluation_metrics_multihead_fl

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.962500,0.037500,0.909465,0.972452,0.907685,0.908440,0.888163,125.000000,137.000000,...,0.918367,0.838710,0.958333,0.941667,0.975000,0.975000,0.943396,0.928814,0.909091,0.852459
1,fold 1,0.983333,0.016667,0.952564,0.988098,0.949742,0.951043,0.950243,126.000000,147.000000,...,0.979592,0.870968,0.977778,0.983333,0.991667,0.980556,0.969231,0.980000,0.969697,0.885246
2,fold 2,0.967967,0.032033,0.903187,0.978015,0.916246,0.909399,0.904645,124.000000,141.000000,...,0.938776,0.833333,0.955432,0.969359,0.980501,0.966574,0.939394,0.962457,0.929293,0.806452
3,fold 3,0.973538,0.026462,0.934898,0.980726,0.926725,0.930565,0.920660,128.000000,143.000000,...,0.836735,0.933333,0.974930,0.966574,0.963788,0.988858,0.966038,0.959732,0.863158,0.933333
4,fold 4,0.984680,0.015320,0.970864,0.987907,0.956807,0.963536,0.953998,126.000000,148.000000,...,0.938776,0.933333,0.980501,0.977716,0.988858,0.991643,0.972973,0.973684,0.958333,0.949153
5,fold 5,0.977716,0.022284,0.924202,0.985738,0.961691,0.941122,0.934097,125.000000,141.000000,...,0.979592,0.966667,0.974930,0.972145,0.983287,0.980501,0.965251,0.965753,0.941176,0.892308
6,fold 6,0.984680,0.015320,0.957821,0.989012,0.957231,0.957321,0.954212,127.000000,146.000000,...,0.979592,0.900000,0.986072,0.977716,0.988858,0.986072,0.980695,0.973333,0.960000,0.915254
7,fold 7,0.976323,0.023677,0.925578,0.983967,0.943316,0.933540,0.929522,126.000000,143.000000,...,0.918367,0.933333,0.972145,0.974930,0.980501,0.977716,0.961832,0.969492,0.927835,0.875000
8,fold 8,0.974930,0.025070,0.925768,0.982874,0.927041,0.926067,0.925398,129.000000,140.000000,...,0.938776,0.838710,0.974930,0.972145,0.977716,0.974930,0.966292,0.965517,0.920000,0.852459
9,fold 9,0.981894,0.018106,0.953089,0.987019,0.942301,0.947333,0.945924,131.000000,142.000000,...,0.938776,0.870968,0.986072,0.974930,0.983287,0.983287,0.981273,0.969283,0.938776,0.900000


<a id="lstm"><a/>

# LSTM

In [6]:
experiment_name  = "LSTM"

model_hyperparameters = {
    "builder": "lstm_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "lstm_units": [256, 256, 128]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 32,
    "verbosity": 0,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
    "callbacks": {
        "early_stop": 5
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_lstm = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [8]:
evaluation_metrics_lstm

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.701389,0.298611,0.0,0.747547,0.250137,0.160258,-0.008338,0.000000,143.000000,...,0.040816,0.00,0.636111,0.402778,0.852778,0.913889,0.000000,0.570858,0.070175,0.000000
1,fold 1,0.741667,0.258333,0.0,0.794242,0.320073,0.261824,0.173150,122.000000,52.000000,...,0.000000,0.00,0.538889,0.650000,0.863889,0.913889,0.595122,0.452174,0.000000,0.000000
2,fold 2,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.00,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.000000
3,fold 3,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.00,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.000000
4,fold 4,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.00,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.000000
5,fold 5,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.00,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.000000
6,fold 6,0.725627,0.274373,0.0,0.766337,0.371644,0.309269,0.086420,0.000000,147.000000,...,0.000000,0.50,0.635097,0.451253,0.863510,0.952646,0.000000,0.598778,0.000000,0.638298
7,fold 7,0.701950,0.298050,0.0,0.764223,0.273259,0.183002,0.055797,130.000000,15.000000,...,0.000000,0.00,0.434540,0.593315,0.863510,0.916435,0.561555,0.170455,0.000000,0.000000
8,fold 8,0.747911,0.252089,0.0,0.798443,0.325665,0.275335,0.189475,114.000000,64.000000,...,0.000000,0.00,0.545961,0.668524,0.863510,0.913649,0.583120,0.518219,0.000000,0.000000
9,fold 9,0.750696,0.249304,0.0,0.802138,0.332770,0.268800,0.203833,131.000000,49.000000,...,0.000000,0.00,0.551532,0.674095,0.863510,0.913649,0.619385,0.455814,0.000000,0.000000


<a id="lstm_fl"><a/>

# LSTM + Focal Loss

In [6]:
experiment_name  = "LSTMFocalLoss"

#Focal Loss
alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

model_hyperparameters = {
    "builder": "lstm_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "lstm_units": [256, 256, 128]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 0,
    "loss": {
        "name": "categorical_focal_loss",
        "parameters": {
            "alpha": alpha,
            "gamma": 2.0
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
    "callbacks": {
        "early_stop": 5
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_lstm_fl = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [9]:
evaluation_metrics_lstm_fl

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.758333,0.241667,0.0,0.810890,0.365790,0.325308,0.241591,129.000000,50.000000,...,0.142857,0.0,0.622222,0.647222,0.850000,0.913889,0.654822,0.440529,0.205882,0.0
1,fold 1,0.841667,0.158333,0.0,0.869492,0.439495,0.384899,0.480046,116.000000,130.000000,...,0.000000,0.0,0.822222,0.766667,0.863889,0.913889,0.783784,0.755814,0.000000,0.0
2,fold 2,0.811978,0.188022,0.0,0.856789,0.477257,0.446387,0.421893,128.000000,75.000000,...,0.428571,0.0,0.768802,0.713092,0.849582,0.916435,0.755162,0.592885,0.437500,0.0
3,fold 3,0.701950,0.298050,0.0,0.772892,0.324622,0.228099,0.098435,130.000000,0.000000,...,0.306122,0.0,0.473538,0.584958,0.832869,0.916435,0.579065,0.000000,0.333333,0.0
4,fold 4,0.777159,0.222841,0.0,0.837842,0.456317,0.391648,0.341103,130.000000,42.000000,...,0.551020,0.0,0.727019,0.665738,0.799443,0.916435,0.726257,0.411765,0.428571,0.0
5,fold 5,0.703343,0.296657,0.0,0.748733,0.255239,0.169149,-0.002320,0.000000,143.000000,...,0.061224,0.0,0.635097,0.406685,0.855153,0.916435,0.000000,0.573146,0.103448,0.0
6,fold 6,0.781337,0.218663,0.0,0.826073,0.372552,0.320436,0.298619,131.000000,70.000000,...,0.020408,0.0,0.626741,0.718663,0.863510,0.916435,0.661616,0.580913,0.039216,0.0
7,fold 7,0.722841,0.277159,0.0,0.778415,0.295661,0.235715,0.111961,118.000000,42.000000,...,0.000000,0.0,0.509749,0.601671,0.863510,0.916435,0.572816,0.370044,0.000000,0.0
8,fold 8,0.792479,0.207521,0.0,0.834603,0.407336,0.375508,0.338948,131.000000,72.000000,...,0.142857,0.0,0.696379,0.688022,0.871866,0.913649,0.706199,0.562500,0.233333,0.0
9,fold 9,0.793872,0.206128,0.0,0.859459,0.518017,0.424962,0.413308,130.000000,42.000000,...,0.795918,0.0,0.830084,0.676880,0.754875,0.913649,0.809969,0.420000,0.469880,0.0


<a id="gru"><a/>

# GRU

In [6]:
experiment_name  = "GRU"

model_hyperparameters = {
    "builder": "gru_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "gru_units": [128, 128]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 32,
    "verbosity": 0,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_gru = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [8]:
evaluation_metrics_gru

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.852778,0.147222,0.0,0.880190,0.455684,0.397545,0.519444,128.000000,126.000000,...,0.000000,0.0,0.830556,0.805556,0.861111,0.913889,0.807571,0.782609,0.000000,0.0
1,fold 1,0.859722,0.140278,0.0,0.886326,0.467959,0.415934,0.543583,130.000000,128.000000,...,0.020408,0.0,0.863889,0.805556,0.855556,0.913889,0.841424,0.785276,0.037037,0.0
2,fold 2,0.860724,0.139276,0.0,0.887572,0.484619,0.453045,0.549527,128.000000,125.000000,...,0.122449,0.0,0.852368,0.807799,0.866295,0.916435,0.828479,0.783699,0.200000,0.0
3,fold 3,0.862117,0.137883,0.0,0.887516,0.465982,0.408433,0.548938,129.000000,131.000000,...,0.000000,0.0,0.857939,0.816156,0.857939,0.916435,0.834951,0.798780,0.000000,0.0
4,fold 4,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.0,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.0
5,fold 5,0.860724,0.139276,0.0,0.886898,0.474577,0.431873,0.546889,129.000000,127.000000,...,0.061224,0.0,0.852368,0.810585,0.863510,0.916435,0.829582,0.788820,0.109091,0.0
6,fold 6,0.863510,0.136490,0.0,0.887939,0.467890,0.408766,0.552460,130.000000,131.000000,...,0.000000,0.0,0.860724,0.813370,0.863510,0.916435,0.838710,0.796353,0.000000,0.0
7,fold 7,0.707521,0.292479,0.0,0.750000,0.250000,0.146654,0.000000,0.000000,149.000000,...,0.000000,0.0,0.635097,0.415042,0.863510,0.916435,0.000000,0.586614,0.000000,0.0
8,fold 8,0.853760,0.146240,0.0,0.881675,0.457551,0.400016,0.524101,130.000000,124.000000,...,0.000000,0.0,0.841226,0.805014,0.855153,0.913649,0.820189,0.779874,0.000000,0.0
9,fold 9,0.863510,0.136490,0.0,0.888240,0.468937,0.410226,0.553466,128.000000,133.000000,...,0.000000,0.0,0.866295,0.813370,0.860724,0.913649,0.842105,0.798799,0.000000,0.0


<a id="gru_fl"><a/>

# GRU + Focal Loss

In [6]:
experiment_name  = "GRUFocalLoss"

#Focal Loss
alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

model_hyperparameters = {
    "builder": "gru_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "gru_units": [128, 128]
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 0,
    "loss": {
        "name": "categorical_focal_loss",
        "parameters": {
            "alpha": alpha,
            "gamma": 2.0
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_gru_fl = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [8]:
evaluation_metrics_gru_fl

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.836111,0.163889,0.0,0.874260,0.466368,0.437882,0.485028,128.000000,105.000000,...,0.183673,0.0,0.833333,0.772222,0.825000,0.913889,0.810127,0.719178,0.222222,0.0
1,fold 1,0.851389,0.148611,0.0,0.880919,0.461316,0.415046,0.519533,130.000000,121.000000,...,0.040816,0.0,0.847222,0.794444,0.850000,0.913889,0.825397,0.765823,0.068966,0.0
2,fold 2,0.852368,0.147632,0.0,0.882341,0.481400,0.454399,0.526491,128.000000,117.000000,...,0.163265,0.0,0.841226,0.791086,0.860724,0.916435,0.817891,0.757282,0.242424,0.0
3,fold 3,0.856546,0.143454,0.0,0.886791,0.476391,0.441679,0.539847,129.000000,122.000000,...,0.102041,0.0,0.863510,0.805014,0.841226,0.916435,0.840391,0.777070,0.149254,0.0
4,fold 4,0.859331,0.140669,0.0,0.892056,0.506910,0.483192,0.557299,128.000000,117.000000,...,0.265306,0.0,0.866295,0.813370,0.841226,0.916435,0.842105,0.777409,0.313253,0.0
5,fold 5,0.841226,0.158774,0.0,0.882520,0.512722,0.482028,0.514116,129.000000,95.000000,...,0.428571,0.0,0.852368,0.771588,0.824513,0.916435,0.829582,0.698529,0.400000,0.0
6,fold 6,0.859331,0.140669,0.0,0.890324,0.503716,0.480454,0.554328,129.000000,117.000000,...,0.244898,0.0,0.863510,0.805014,0.852368,0.916435,0.840391,0.769737,0.311688,0.0
7,fold 7,0.856546,0.143454,0.0,0.885622,0.490319,0.465054,0.540201,130.000000,117.000000,...,0.183673,0.0,0.844011,0.799443,0.866295,0.916435,0.822785,0.764706,0.272727,0.0
8,fold 8,0.845404,0.154596,0.0,0.878350,0.461067,0.422850,0.505559,130.000000,114.000000,...,0.081633,0.0,0.841226,0.788301,0.838440,0.913649,0.820189,0.750000,0.121212,0.0
9,fold 9,0.850975,0.149025,0.0,0.885409,0.494908,0.469455,0.531692,129.000000,111.000000,...,0.244898,0.0,0.860724,0.791086,0.838440,0.913649,0.837662,0.747475,0.292683,0.0


<a id="transformer"><a/>

# Time-Series Transformer

In [15]:
experiment_name  = "Transformer"

#Focal Loss
#alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

model_hyperparameters = {
    "builder": "transformer_builder",
    "input_shape": (6000,1),
    "num_classes":4
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 0,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {}
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_transformer = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [19]:
evaluation_metrics_transformer

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.962500,0.037500,0.902309,0.973223,0.907685,0.904586,0.888455,125.00000,137.000000,...,0.918367,0.838710,0.969444,0.938889,0.966667,0.975000,0.957854,0.925676,0.882353,0.852459
1,fold 1,0.970833,0.029167,0.929400,0.978534,0.914327,0.921545,0.912623,127.00000,143.000000,...,0.857143,0.870968,0.980556,0.955556,0.963889,0.983333,0.973180,0.947020,0.865979,0.900000
2,fold 2,0.949861,0.050139,0.874939,0.964273,0.871156,0.872442,0.850365,127.00000,132.000000,...,0.795918,0.833333,0.963788,0.922006,0.935933,0.977716,0.951311,0.904110,0.772277,0.862069
3,fold 3,0.972145,0.027855,0.933961,0.979779,0.915160,0.923911,0.916412,128.00000,143.000000,...,0.857143,0.866667,0.977716,0.963788,0.961003,0.986072,0.969697,0.956522,0.857143,0.912281
4,fold 4,0.976323,0.023677,0.952255,0.981721,0.926619,0.938661,0.928687,127.00000,146.000000,...,0.857143,0.900000,0.977716,0.966574,0.972145,0.988858,0.969466,0.960526,0.893617,0.931034
5,fold 5,0.966574,0.033426,0.913445,0.975643,0.921106,0.916486,0.900071,126.00000,140.000000,...,0.816327,0.966667,0.980501,0.941504,0.958217,0.986072,0.972973,0.930233,0.842105,0.920635
6,fold 6,0.981894,0.018106,0.957735,0.986772,0.950183,0.952912,0.945863,125.00000,146.000000,...,1.000000,0.866667,0.974930,0.977716,0.988858,0.986072,0.965251,0.973333,0.960784,0.912281
7,fold 7,0.970752,0.029248,0.917704,0.978916,0.912983,0.915047,0.912375,125.00000,144.000000,...,0.897959,0.833333,0.966574,0.966574,0.980501,0.969359,0.954198,0.960000,0.926316,0.819672
8,fold 8,0.974930,0.025070,0.933125,0.981852,0.929334,0.930941,0.925114,128.00000,142.000000,...,0.877551,0.903226,0.980501,0.963788,0.974930,0.980501,0.973384,0.956229,0.905263,0.888889
9,fold 9,0.980501,0.019499,0.960308,0.985210,0.924448,0.940411,0.941415,131.00000,144.000000,...,0.918367,0.806452,0.980501,0.977716,0.983287,0.980501,0.973978,0.972973,0.937500,0.877193


<a id="transformer_fl"><a/>

# Time-Series Transformer + Focal Loss

In [6]:
experiment_name  = "TransformerFocalLoss"

#Focal Loss
alpha = np.array([1.0 - 0.3647, 1.0 - 0.4143, 1.0 - 0.1364, 1.0 - 0.0843])

model_hyperparameters = {
    "builder": "transformer_builder",
    "input_shape": (6000,1),
    "num_classes":4
}

training_configuration = {
    "epochs": 200,
    "batch_size": 64,
    "verbosity": 0,
    "loss": {
        "name": "categorical_focal_loss",
        "parameters": {
            "alpha": alpha,
            "gamma": 2.0
        }
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.0001
        }
    },
    "callbacks": {
        "early_stop": 10
    }     
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics_transformer_fl = k_fold_cross_validation(
    project, experiment_name, dataset_artifact_name, labels,
    model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234
)

In [8]:
evaluation_metrics_transformer_fl

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.969444,0.030556,0.903239,0.979761,0.935696,0.918019,0.909770,125.00000,138.000000,...,0.959184,0.903226,0.977778,0.952778,0.975000,0.972222,0.968992,0.941980,0.912621,0.848485
1,fold 1,0.962500,0.037500,0.886121,0.976509,0.921381,0.899797,0.890015,127.00000,132.000000,...,0.959184,0.870968,0.980556,0.944444,0.950000,0.975000,0.973180,0.929577,0.839286,0.857143
2,fold 2,0.961003,0.038997,0.902097,0.972073,0.901008,0.900953,0.883630,124.00000,138.000000,...,0.897959,0.833333,0.961003,0.944290,0.961003,0.977716,0.946565,0.932432,0.862745,0.862069
3,fold 3,0.965181,0.034819,0.916272,0.974689,0.902961,0.909320,0.895461,128.00000,141.000000,...,0.755102,0.933333,0.974930,0.952646,0.941504,0.991643,0.966038,0.943144,0.778947,0.949153
4,fold 4,0.976323,0.023677,0.936381,0.983499,0.946049,0.940725,0.929453,123.00000,145.000000,...,0.938776,0.933333,0.972145,0.972145,0.972145,0.988858,0.960938,0.966667,0.901961,0.933333
5,fold 5,0.965181,0.034819,0.910407,0.974453,0.909348,0.909170,0.895642,126.00000,141.000000,...,0.795918,0.933333,0.980501,0.941504,0.955432,0.983287,0.972973,0.930693,0.829787,0.903226
6,fold 6,0.984680,0.015320,0.967570,0.988291,0.963886,0.965610,0.954168,127.00000,145.000000,...,0.979592,0.933333,0.983287,0.972145,0.991643,0.991643,0.976923,0.966667,0.969697,0.949153
7,fold 7,0.963788,0.036212,0.893862,0.974883,0.904594,0.899061,0.892139,125.00000,139.000000,...,0.897959,0.833333,0.966574,0.952646,0.966574,0.969359,0.954198,0.942373,0.880000,0.819672
8,fold 8,0.969359,0.030641,0.906996,0.979472,0.934901,0.919865,0.909393,124.00000,139.000000,...,0.918367,0.935484,0.969359,0.961003,0.969359,0.977716,0.957529,0.952055,0.891089,0.878788
9,fold 9,0.973538,0.026462,0.926415,0.981380,0.918526,0.922190,0.921010,129.00000,141.000000,...,0.897959,0.838710,0.983287,0.963788,0.969359,0.977716,0.977273,0.955932,0.888889,0.866667


<a id="references"><a/>

# References

* [1] In-depth comparison of deep artificial neural network architectures on seismic events classification
* [2] Llaima volcano dataset: In-depth comparison of deep artificial neural network architectures on seismic events classification